# 1.実行環境の整備

## 1.1.「12時間ルール」および「90分ルール」

学習処理に長時間を要する場合の対策を示す。<br>
Google Colaboratoryでは、以下のいずれかの条件を満たす場合、実行中のプログラムがあってもインスタンスの状態がすべてリセットされる。<br>
* 【12時間ルール】新しいインスタンスを起動してから12時間経過
* 【90分ルール】ノートブックのセッションが切れてから90分経過<br>

参考：https://qiita.com/enmaru/items/2770df602dd7778d4ce6

なお、本対策を導入した場合でも、「バックエンドを割り当てられませんでした」により、GPUが使用不可になる現象には対応できない。<br>
参考：https://qiita.com/canonrock16/items/5b4d70150d70a5af9339

### 1.1.1.「90分ルール」対策

まず、90分ルール対策としては、定期的にノートブックの更新を実施する方法が有効である。<br>
ノートブックの自動更新は、Google Chromeのアドイン「Auto Refresh」を用いて実現できる。まずはこの「Auto Refresh」をインストールする。<br>

しかし、セル実行中にノートブックの更新を実施しようとすると、サイトの再読み込みが必須となるため、ノートブックの更新時に毎回、ブラウザの「再読み込み」を選択する必要がある。<br>
この状態では、「Auto Refresh」によって自動更新を実施することができない。<br>

そこで、Google Colaboratoryのメニューの、<br>
「ランタイム」→「ランタイムのタイプを変更」から、<br>
「このノートブックを保存する際にコードセルの出力を除外する」にチェックを入れる。<br>
この設定により、ページの更新時に毎回「再読み込み」を選択する必要がなくなる。<br>
(初回更新時のみ手動で「再読み込み」を選択する必要がある。)<br>
ただし、代わりに、各セルの実行結果がノートブックに保存されなくなる。<br>

### 1.1.2.「12時間ルール」対策

「90分ルール」対策により、各セルの実行結果がノートブックに保存されない状態になっている。<br>
この状態では、長時間かけて学習した際の、各セルの実行結果がノートブックに残らないため、90分ルール対策を講じた場合でも、12時間経過してしまうと、学習過程や結果の出力が確認できなくなってしまう。<br>
そこで、「12時間ルール」対策として、各セルの実行結果を、標準出力ではなくファイルに出力する設定を行う。<br>
この設定により、セルの実行結果がGoogle Drive上のファイルに記録されるため、12時間が経過した場合でも、実行された時点までの出力結果を確認することができる。<br>
※12時間以内に実行結果を確認可能な場合は、「90分ルール」対策の「Auto Refresh」の導入だけでよい。<br>

## 1.2.実施手順

* Google Colaboratoryのメニュー「ランタイム」→「ランタイムのタイプを変更」から、<br>
「このノートブックを保存する際にコードセルの出力を除外する」にチェックを入れる。
* Google Chromeのアドイン「Auto Refresh」をインストールする。
* Google CholaboratoryにGoogle Driveをマウントする。(12時間ルール対策のみ)
* セルの実行結果の出力先を、標準出力から、Google Driveの任意のファイルに変更する。(12時間ルール対策のみ)
* 必要な各処理のセル、および学習処理のセル(処理に時間がかかるセル)を実行する。
* 学習処理が開始したら、「Auto Refresh」を開き、任意の時間間隔(10min等)をセットして「Start」する。<br>
(最初の1ループは待機する必要があるため、待てる時間をセットするとよい)
* 指定した時間が経過すると、ブラウザに「このサイトを再読み込みしますか？」が表示されるので、「再読み込み」を選択する。
* ノートブックが再読み込みされ、自動的に学習処理のセルの実行が再開したら、そのまま放置する。

## 1.3.実行コード

In [0]:
# Google Colaboratoryに、Google Driveをマウントする。(12時間ルール対策のみ)
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
# セル実行結果の出力先を、標準出力からGoogle Drive上のテキストファイルに変更する。(12時間ルール対策のみ)
%%time
import sys
sys.stdout = open('/content/drive/My Drive/tmp/result_12_pytorch-ResNet50_CIFAR-10_Aug-2.txt', 'w')
# 念のため、ノートブックの末尾で、セルの実行結果の出力先を標準出力に戻す処理を入れておく。

参考：https://alicehimmel.hatenadiary.org/entry/20110213/1297611415

# 2.ライブラリの準備

Pytorchのライブラリ(lib)をzip形式でGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=1TrZJ2KnlMLXg1-P8XQBnTISeQ4icmhS4<br>
取得したリンクからidを抜き出す。<br>
**1TrZJ2KnlMLXg1-P8XQBnTISeQ4icmhS4**

In [0]:
# Google Cloud SDKを用いて、Google Drive上のファイルを、Google Colabolatoryの一時領域にダウンロードするための認証
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

参考：https://qiita.com/uni-3/items/201aaa2708260cc790b8

In [0]:
# Pytorchのライブラリをダウンロード
id = '1TrZJ2KnlMLXg1-P8XQBnTISeQ4icmhS4' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
import os
os.chdir('/content')
downloaded.GetContentFile('lib.zip')

In [0]:
# ダウンロードを確認
!pwd
%ls

In [0]:
# ライブラリを解凍
%%time
import zipfile
import os
os.chdir('/content')
save_name = "lib"
save_path = save_name + '.zip'
with zipfile.ZipFile(save_path) as zip: # ZIPファイルを読み込み/自動クローズ
  zip.extractall() # ZIPファイルを解凍
os.remove(save_path) # ZIPファイルを消去

In [0]:
# 処理結果を確認
!pwd
!ls
%ls ./lib/

# 3.データセットの準備

## 3.1.データセットを自分で用意する場合

### 3.1.1.データセットの入手/格納/展開

データセットを入手する。<br>
https://www.vision.ee.ethz.ch/datasets_extra/food-101/

ダウンロードしたデータセットをGoogle Driveにアップロードする。<br>
「共有可能なリンクを取得」でリンクを取得する。<br>
https://drive.google.com/open?id=1qWuEvaweWvGN2g6ylpfBQgpasXfyKSuO<br>
取得したリンクからidを抜き出す。<br>
**1qWuEvaweWvGN2g6ylpfBQgpasXfyKSuO**

In [0]:
# データセットをダウンロード
%%time
id = '1qWuEvaweWvGN2g6ylpfBQgpasXfyKSuO' # 事前に取得したid("id="の後ろの部分)を代入
downloaded = drive.CreateFile({'id': id})
os.chdir('/content')
downloaded.GetContentFile('food-101.tar.gz') # データセットのファイル名を入力

In [0]:
# ダウンロードを確認
!pwd
%ls

In [0]:
# データセットを解凍(.zipの場合)
'''
%%time
import zipfile
import os
os.chdir('/content')
save_name = "find-a-car-park" # データセットのファイル名を入力
save_path = save_name + '.zip'
with zipfile.ZipFile(save_path) as zip: # ファイル読み込み/自動クローズ
  zip.extractall(save_name) # .zipファイルを解凍
#os.remove(save_path) # .zipファイルを消去 # 試験運用中は消さないでおく
'''

In [0]:
# データセットを解凍(.tar.gzの場合)
%%time
import tarfile
import os
os.chdir('/content')
save_name = "food-101" # データセットのファイル名を入力
save_path = save_name + '.tar.gz'
with tarfile.open(save_path) as tar: # ファイル読み込み/自動クローズ
  tar.extractall(save_name) # .tar.gzファイルを解凍
#os.remove(save_path) # .tar.gzファイルを消去 # 試験運用中は消さないでおく

In [0]:
# 処理結果を確認
!ls ./food-101/food-101/
%ls ./food-101/food-101/images

In [0]:
# データ個数を確認
!ls -1U "/content/food-101/food-101/images/apple_pie" | wc -l
!ls -1U "/content/food-101/food-101/images/waffles" | wc -l

### 3.1.2.train/validationデータの分割

In [0]:
os.chdir('/content/food-101/food-101/')
!pwd
%ls

In [0]:
# trainデータ、valデータを格納するフォルダを作成
os.mkdir("/content/food-101/food-101/train/")
os.mkdir("/content/food-101/food-101/val/")
!pwd
%ls

In [0]:
# フォルダが空であることを確認
!ls "/content/food-101/food-101/train/"
!ls -1U "/content/food-101/food-101/train/" | wc -l
!ls "/content/food-101/food-101/val/"
!ls -1U "/content/food-101/food-101/val/" | wc -l

In [0]:
#「/content/find-a-car-park/data/」以下の画像を「train」「val」フォルダに振り分ける
# train:val = 3:1
import glob
import shutil
os.chdir('/content/food-101/food-101/images')
label_dirs = os.listdir(path='.')
path = os.getcwd()  

# ラベル名が付与されている各フォルダごとに、中身のファイル名を取り出す
for dir_name in label_dirs:
  files = os.listdir(path + "/" + dir_name) # 1個のラベルフォルダのパスを取得
  numof_data = len(files) # ラベルフォルダ中のデータ数を取得
  files = glob.glob("./" + dir_name + "/*") # ラベルフォルダ中の全ファイルリストを取得

  # ラベルフォルダ中のファイルを抽出して、"val"フォルダに移動する
  # 全体の1/4のデータを"val"フォルダに移動するまで繰り返す
  i = 0
  print("Number of data in " + dir_name + ": " + str(numof_data))
  for file in files:
    i = i + 1
    if i < numof_data * 1/4:
      data_dir = "/content/food-101/food-101/val/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
    else:
      data_dir = "/content/food-101/food-101/train/" + dir_name
      if not os.path.exists(data_dir):
        os.mkdir(data_dir)
      shutil.move(file, data_dir)
  shutil.rmtree("./" + dir_name) # データを振り分け終えたラベルフォルダは削除しておく

In [0]:
# データが振り分けられたことを確認
!ls -1U "/content/food-101/food-101/train/apple_pie" | wc -l
!ls -1U "/content/food-101/food-101/train/waffles" | wc -l

In [0]:
# データが振り分けられたことを確認
!ls -1U "/content/food-101/food-101/val/apple_pie" | wc -l
!ls -1U "/content/food-101/food-101/val/waffles" | wc -l

In [0]:
# [Option] データセットを一旦消す場合に実行
#%rm -r '/content/food-101/food-101/'

## 3.2.データセットをライブラリからダウンロードする場合

学習直前のData Loaderの設定時にデータセットをダウンロードする。

# 4.学習

## 4.1.前準備

In [0]:
%%time
os.chdir('/content/')
!npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

In [0]:
%%time
import time
time.sleep(5)
! cat url.txt

In [0]:
!pwd
!ls

In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

## 4.2.アーキテクチャの選択

In [0]:
# アーキテクチャを選択して使用する。
%%time
#net = network.VGG16(class_size=10)
#net = network.InceptionV3(class_size=10)
net = network.ResNet50(class_size=10)

## 4.3.データローダの設定

### 4.3.1.データセットを自分で用意した場合

In [0]:
# Data Loaderの設定 (自分で用意したデータセットを使用する場合)
# Data Augmentation無しの場合
'''
%%time
batch_size = 128
train_dataset = torchvision.datasets.ImageFolder(root='/content/food-101/food-101/train/', transform=dl.simple_transform(resize=224)) # trainデータのフォルダを指定
test_dataset = torchvision.datasets.ImageFolder(root='/content/food-101/food-101/val/', transform=dl.simple_transform(resize=224)) # valデータのフォルダを指定
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size, suffle=True)
'''

In [0]:
# Data Loaderの設定 (自分で用意したデータセットを使用する場合)
# Data Augmentation有りの場合
'''
%%time
batch_size = 128
# 各Augmentationの有無を指定
transform_train = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=True, Rotation=True, Perspective=True, Crop=True, Erasing=True)
transform_val = dl.val_transform()
train_dataset = torchvision.datasets.ImageFolder(root='/content/food-101/food-101/train/', transform=transform_train) # trainデータのフォルダを指定
test_dataset = torchvision.datasets.ImageFolder(root='/content/food-101/food-101/val/', transform=transform_val) # valデータのフォルダを指定
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size, suffle=True, num_workers=2)
'''

In [0]:
# [Option] フォルダを一旦消す場合に実行
#!rm -rf "/content/food-101/food-101/train/"
#!rm -rf "/content/food-101/food-101/val"

### 4.3.2.データセットをライブラリからダウンロードする場合

In [0]:
# Data Loaderの設定 (CIFAR-10をダウンロードして使用する場合)
# Data Augmentation無しの場合
'''
%%time
batch_size = 128
train_dataset = torchvision.datasets.CIFAR10(root='/content/cifar-10/train/', train=True, download=True, transform=dl.simple_transform(resize=224))
test_dataset = torchvision.datasets.CIFAR10(root='/content/cifar-10/val/', train=False, download=True, transform=dl.simple_transform(resize=224))
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size, suffle=True)
'''

In [0]:
# Data Loaderの設定 (CIFAR-10をダウンロードして使用する場合)
# Data Augmentation有りの場合
%%time
batch_size = 128
# 各Augmentationの有無を指定
transform_train = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=False, Rotation=True, Perspective=True, Crop=True, Erasing=True)
transform_val = dl.val_transform()
train_dataset = torchvision.datasets.CIFAR10(root='/content/cifar-10/train/', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='/content/cifar-10/val/', train=False, download=True, transform=transform_val)
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size, suffle=True)

## 4.4.モデル定義/学習実行

In [0]:
# モデル定義
%%time
model = architecture.CNN_Architecture(net)

In [0]:
# 学習処理
%%time
model.train(data_loader, epoch_count=50, is_inception=False) #, optimizer=optimizer.SGD(net))

# 5.検証

In [0]:
# テストデータで性能検証 (ただし現状はtestデータ=validationデータ)
%%time
model.predict(data_loader['val'])

# 6.後処理

In [0]:
# セル実行結果の出力先を、Google Drive上のテキストファイルから標準出力に変更する。 (12時間ルール対策終了)
sys.stdout.close()
sys.stdout = sys.__stdout__

------------------